# Import Library

In [27]:
import pandas as pd
import duckdb
import os

# Make eklipse_data.db

In [43]:
# Inisialisasi koneksi DuckDB (in-memory atau file)
con = duckdb.connect(database=':memory:')  # Atau ganti dengan 'database.duckdb' kalau ingin persist

# Load semua CSV sebagai view (DuckDB bisa baca langsung dari disk!)
con.execute("""
    CREATE VIEW downloaded_clips AS SELECT * FROM read_csv_auto('da_test/downloaded_clips.csv');
    CREATE VIEW shared_clips AS SELECT * FROM read_csv_auto('da_test/shared_clips.csv');
    CREATE VIEW clips AS SELECT * FROM read_csv_auto('da_test/clips.csv');
    CREATE VIEW premium_users AS SELECT * FROM read_csv_auto('da_test/premium_users.csv');
    CREATE VIEW gamesession AS SELECT * FROM read_csv_auto('da_test/gamesession.csv');
""")

## SQL

## Question SQL 1
The total number of users who have downloaded any clips. total number of clips downloaded. The total number of game sessions from which the downloaded clips were generated. 

In [44]:
query = """--sql 
SELECT
    COUNT(DISTINCT user_id) AS total_user_who_download_clips,
    COUNT(DISTINCT clip_id) AS total_clips_downloaded,
    COUNT(DISTINCT gamesession_id) AS total_gamesession_generated
FROM downloaded_clips dc;"""

df = con.execute(query).fetchdf()
df


,total_user_who_download_clips,total_clips_downloaded,total_gamesession_generated
0,1454,12935,4338


## Question SQL 2
For users who purchased premium in the last 3 months: Get the number of 
users who purchased premium, the number of users who shared any clips, 
the total number of clips shared, and the total number of gamesessions from 
which the shared clips were generated. 

In [45]:
query = """--sql 
WITH premiums_users_last_3_month AS (
    SELECT user_id
    FROM premium_users
    WHERE CAST(starts_at AS TIMESTAMP) >= (
        SELECT MAX(CAST(starts_at AS TIMESTAMP)) - INTERVAL '3 MONTH'
        FROM premium_users
    )
),
premium_summary AS (
    SELECT 
        COUNT(DISTINCT user_id) AS total_user_who_purchased_premium_last_3_months
    FROM premiums_users_last_3_month
),
shared_clips_summary AS (
    SELECT 
        COUNT(DISTINCT sc.user_id) AS number_of_user_premium_last_3_month_who_shared_clips,
        COUNT(DISTINCT sc.clip_id) AS number_of_clip_shared,
        COUNT(DISTINCT sc.gamesession_id) AS number_of_gamesession_generated_from_shared_clips
    FROM shared_clips sc
    JOIN premiums_users_last_3_month pul3m ON sc.user_id = pul3m.user_id
)

SELECT 
    ps.total_user_who_purchased_premium_last_3_months,
    scs.number_of_user_premium_last_3_month_who_shared_clips,
    scs.number_of_clip_shared,
    scs.number_of_gamesession_generated_from_shared_clips
FROM premium_summary ps
CROSS JOIN shared_clips_summary scs;

"""

df = con.execute(query).fetchdf()
df


,total_user_who_purchased_premium_last_3_months,number_of_user_premium_last_3_month_who_shared_clips,number_of_clip_shared,number_of_gamesession_generated_from_shared_clips
0,164,59,1144,359


## Question SQL 3
Calculate, on a weekly basis: The number of users engaged. The number of 
clips engaged. The total number of gamesessions from which the engaged 
clips were generated.

In [46]:
query = """--sql 
WITH premiums_users_last_3_month AS (
    SELECT user_id
    FROM premium_users
    WHERE starts_at >= (
        SELECT MAX(starts_at) - INTERVAL '3 MONTH'
        FROM premium_users
    )
),
premium_summary AS (
    SELECT 
        COUNT(DISTINCT user_id) AS total_user_who_purchased_premium_last_3_months
    FROM premiums_users_last_3_month
),
shared_clips_summary AS (
    SELECT 
        COUNT(DISTINCT sc.user_id) AS number_of_user_premium_last_3_month_who_shared_clips,
        COUNT(DISTINCT sc.clip_id) AS number_of_clip_shared,
        COUNT(DISTINCT sc.gamesession_id) AS number_of_gamesession_generated_from_shared_clips
    FROM shared_clips sc
    JOIN premiums_users_last_3_month pul3m ON sc.user_id = pul3m.user_id
)

SELECT 
    ps.total_user_who_purchased_premium_last_3_months,
    scs.number_of_user_premium_last_3_month_who_shared_clips,
    scs.number_of_clip_shared,
    scs.number_of_gamesession_generated_from_shared_clips
FROM premium_summary ps
CROSS JOIN shared_clips_summary scs;
"""

df = con.execute(query).fetchdf()
df


,total_user_who_purchased_premium_last_3_months,number_of_user_premium_last_3_month_who_shared_clips,number_of_clip_shared,number_of_gamesession_generated_from_shared_clips
0,164,59,1144,359


# B. Data Analysis

In [63]:
query_gamesession_premium_users = """--sql
-- query premium user ids
with gamession_with_premium_users as (
-- query gamesession who the user_id is premium
SELECT 
    *
FROM
    gamesession
WHERE
    user_id IN (SELECT DISTINCT user_id from premium_users)
)

select 
    count(distinct id) as total_gamesession_generated_by_premium_users,
    count(distinct user_id) as total_premium_users_who_generated_gamesession 
from 
    gamession_with_premium_users

"""

query_gamesession_free_users = """--sql
-- query free user ids
with gamession_with_free_users as (
-- query gamesession who the user_id is free
SELECT 
    *
FROM
    gamesession
WHERE
    user_id NOT IN (SELECT DISTINCT user_id from premium_users)
)

select 
    count(distinct id) as total_gamesession_generated_by_free_users,
    count(distinct user_id) as total_free_users_who_generated_gamesession 
from 
    gamession_with_free_users

"""

df_premium_users = con.execute(query_gamesession_premium_users).fetchdf()
df_free_users = con.execute(query_gamesession_free_users).fetchdf()

df_free_users

,total_gamesession_generated_by_free_users,total_free_users_who_generated_gamesession
0,28524,4002


In [64]:
df_premium_users

,total_gamesession_generated_by_premium_users,total_premium_users_who_generated_gamesession
0,9055,320
